In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
from read_openmeteo_hour import read_openmeteo

meta = pd.read_csv('metadata5000best.csv')
data = pd.read_csv('stations_complete_2016_2019_1h.csv')


meta = meta.set_index('Unnamed: 0')
meta = meta.drop('Unnamed: 0.1', axis = 1)
meta.index.name = 'stacja'
data = data.set_index('date')
data.index.name = 'date'

data_long = data.reset_index().melt(id_vars = 'date', var_name = 'stacja', value_name = 'pm25')
data_long["date"] = pd.to_datetime(data_long["date"], utc=True, format='mixed')
data_long['date'] =data_long['date'].dt.round('H')

data_long

In [ ]:
import time

meta2 = meta.copy()


all_weather = []
elevations = []


for stacja, lon, lat in zip(meta.index, meta['lon'], meta['lat']):
    print(f"Pobieram pogodę dla {stacja} ({lat}, {lon})...")
    
    df_weather, elevation = read_openmeteo(lon, lat)
    
    elevations.append(elevation)
    
    if df_weather.index.name == "date" or df_weather.index.dtype.kind == "M":
        df_weather = df_weather.reset_index()
    
    df_weather["stacja"] = stacja
    all_weather.append(df_weather)
    
    print("Wait 15 seconds before next request...")
    time.sleep(6)

meta2["elevation"] = elevations

df_weather_all = pd.concat(all_weather, ignore_index=True)

df_weather_all["date"] = pd.to_datetime(df_weather_all["date"], utc=True)

df_weather_all["date"] = df_weather_all["date"].dt.tz_convert('Europe/Warsaw')


df_full = pd.merge(data_long, df_weather_all, on=["date", "stacja"], how="inner")

print(meta2)
print(df_full)

In [ ]:

df_full.to_csv('stations_weather_2016_2019_1h.csv')
meta2.to_csv('meta_elevation.csv')